In [ ]:
import pandas as pd
import numpy as np
import sqlite3

## Создаю соединение с файлом БД

In [ ]:
con = sqlite3.connect('.\content\indicators_db.sqlite3')

Создаю функциональную обертку для удобства запросов в БД

In [ ]:
def read_sql(sql_query, connection=con):
    return pd.read_sql(sql= sql_query, con=connection)

В переменную буду передавать запросы

In [ ]:
sql_q = 'select * from main.org_indicators_2020_2022'

In [ ]:
df= read_sql(sql_q)
df.head()

# Загрузка дополнительных таблиц

In [ ]:
df_main = pd.read_csv('./content/main.csv')
df_net_property = pd.read_csv('./content/net_property.csv')
df_seats_property = pd.read_csv('./content/seats_property.csv')
df_type_property = pd.read_csv('./content/type_property.csv')
# Актуализирую таблицу с посадочными местами
df_seats_property = df_seats_property.sort_values(by=['ID', 'ver_ID'], ascending=[True, False]) \
    .drop_duplicates('ID', keep='first')

# Задание 1.
<i>Для каждой организации из main.csv подтяните выручку и число сотрудников за 2020, 2021 и 2022. Для каждой организации из main.csv подтяните выручку и число сотрудников за 2020, 2021 и 2022. Посчитайте дельту по выручке и сотрудникам между 2022 и 2020. Отсортируйте организации по убыванию выручки. Для организаций с одинаковой выручкой выполните сортировку по возрастанию ID. Если у организации нет данных по выручке или числу сотрудников за 2020 или 2022, то исключите их. Названия колонок сделайте на русском языке.</i>

##### В SQL запросе сразу провожу агрегацию VALUE группировкой по колонкам  ID YEAR INDICATOR_NAME. Фильтрую INDICATOR_NAME  по Выручке и числу сотрудников

In [ ]:
sql_q = '''select ID, YEAR, INDICATOR_NAME, SUM(VALUE) as VALUE
from main.org_indicators_2020_2022
where INDICATOR_NAME in ('Выручка', 'Число сотрудников')
group by ID, YEAR, INDICATOR_NAME;'''
# sql_q = '''select *
# from main.org_indicators_2020_2022
# where INDICATOR_NAME in ('Выручка', 'Число сотрудников');'''
df = read_sql(sql_q)
df.head(20)

In [ ]:
t1 = df.pivot_table(index='ID', columns=['INDICATOR_NAME', 'YEAR'], values='VALUE')
t1.head()

##### Преобразование имен колонок. Убираю мультииндекс

In [ ]:
t1.columns = [x+' '+str(y) for x, y in t1.columns]
t1.head()

In [ ]:
t1.reset_index(inplace=True)

Объединяю 2 датафрейма (Main и t1)

In [ ]:
df1 = df_main.merge(t1, on='ID')
df1.head()

Считаю дельты по годам. Вычитаю из года более позднего (2022), год более ранний (2020)

In [ ]:
df1['Дельта по выручке'] = df1['Выручка 2022'] - df1['Выручка 2020']
df1['Дельта по сотрудникам'] = df1['Число сотрудников 2022'] - df1['Число сотрудников 2020']

Переименование столбцов на русский язык.

In [ ]:
df1 = df1.rename(columns={'ID': 'Идентификатор', 'Name': 'Наименование', 'Address': 'Адрес'})
df1 = df1[['Идентификатор', 'Наименование', 'Адрес', 'Выручка 2020',
       'Выручка 2021', 'Выручка 2022', 'Дельта по выручке', 'Число сотрудников 2020',
       'Число сотрудников 2021', 'Число сотрудников 2022',
       'Дельта по сотрудникам']]
df1.head()

Сортировка по заданию, удаление NaN по заданию.

In [ ]:
df1 = df1.sort_values(by=['Дельта по выручке', 'Идентификатор'], ascending=[False, True])
df_result = df1.dropna(subset=['Выручка 2020', 'Выручка 2022', 'Число сотрудников 2020', 'Число сотрудников 2022'])
df_result.head()

Сохранение результатов в CSV файл

In [ ]:
df_result.to_csv('ex_6_1_1.csv', index=False)

In [ ]:
df_old = pd.read_csv('ex_6_1.csv')

In [ ]:
print((df_result.values == df_old.values))

# Задание 2
Коллеги уже пишут вам в чат и просят посчитать суммарную выручку по организациям за 2022 год:
357706, 198181, 27549
Через несколько минут приходит еще одно сообщение, просят добавить к ним еще четыре:
217895, 201139, 357706, 20105
Вы почти успели импортировать pandas и коллеги снова просят добавить к ним еще организаций из списка.
Посчитайте, пожалуйста, суммарную выручку за 2022 год. Если у каких-то организаций нет выручки за 2022 год, то возьмите самую свежую доступную (за другой год). Ответ округлите до целого.

In [ ]:
id_lst = pd.read_excel('./content/Выручка_по_компаниям.xlsx', sheet_name=1)
id_lst = id_lst.rename(columns={id_lst.columns[0]: 'ID'})
n_id = pd.DataFrame({'ID': [357706, 198181, 27549, 217895, 201139, 357706, 20105]})
id_lst = pd.concat([id_lst, n_id]).set_index(np.arange(len(id_lst) + len(n_id)))

In [ ]:
sql_q = f'''select ID, YEAR, VALUE from org_indicators_2019
where INDICATOR_NAME='Выручка'
and ID in {tuple(id_lst['ID'].values)}
union
select ID, YEAR, VALUE from main.org_indicators_2020_2022
where INDICATOR_NAME='Выручка'
and ID in {tuple(id_lst['ID'].values)}'''

In [ ]:
df = read_sql(sql_q)

In [ ]:
df = df.sort_values(by=['ID', 'YEAR'], ascending=[True, False]).drop_duplicates('ID', keep='first')
print(df.VALUE.sum())